## Notebook used for constructing the datasets used in the empirical section

### 1) historical Standard & Poors 500 constituents dataset

In [46]:
import pickle
import os 
with open("fmp_key.pkl", "rb") as input_file:
  fmp_key = pickle.load(input_file)

with open("eod_key.pkl", "rb") as input_file:
  eod_key = pickle.load(input_file)

import urllib, json
import requests
from urllib.request import urlopen
import pandas as pd

# Function to get the Data

def get_jsonparsed_data(url):
    res = urlopen(url)
    data = res.read().decode("utf-8")
    return json.loads(data)

import datetime as dt
from tqdm import tqdm 

filename = 'price_df.csv'
data_provider  = 'eod' # other choice is 'fmp'

# first look up the historical s&p 500 constituents
constituents = get_jsonparsed_data("https://financialmodelingprep.com/api/v3/historical/sp500_constituent?apikey={}".format(fmp_key))
constituents = pd.DataFrame(constituents)
constituents['date'] = pd.to_datetime(constituents['date'])
all_tickers = set(constituents['symbol'].values)

begin = dt.datetime(1980,1,2)
end   = dt.datetime(2024,1,1)

price_df = pd.DataFrame(index=pd.date_range(start=begin, end=end, freq="B"))
failed_to_add = list()
# check if the file already exists
if os.path.exists(filename):
    price_df = pd.read_csv(filename)
    price_df = price_df.rename(columns={'Unnamed: 0':'date'})
    price_df.set_index('date', inplace=True)    

for count, ticker in enumerate(tqdm(all_tickers)):
    # find the correct dates the stock was part of the s&p 500
    date_begin = begin
    date_end   = end
    ticker = ticker.split(" ")[0]
    
    if ticker in price_df.columns:
        continue
    for _, row in constituents[constituents.symbol == ticker].iterrows():
        if len(row['addedSecurity']) != 0:
            date_begin = begin if row.date <= begin else row.date
        if len(row['removedSecurity']) != 0:
            date_end = end if row.date >= end else end

    if data_provider == 'fmp':
        # get the data from the fmp api
        df = get_jsonparsed_data("https://financialmodelingprep.com/api/v3/historical-price-full/{}?from={}&to={}&apikey={}".format(ticker,
            date_begin.strftime('%Y-%m-%d'),date_end.strftime('%Y-%m-%d'),fmp_key))
        if len(df) == 0:
            print('could not add {}'.format(ticker),end="")
            failed_to_add.append(ticker)
            continue
        df1 = pd.DataFrame(df['historical'])
        df1['date'] = pd.to_datetime(df1['date'])
        df1.set_index('date',inplace=True)
        # merge the dataset
        price_df = pd.merge(price_df, df1['adjClose'], how='left', left_index=True, right_index=True)
        price_df = price_df.rename(columns={'adjClose': ticker})
    elif data_provider == 'eod':
        df  = get_jsonparsed_data("https://eodhd.com/api/eod/{}.US?from={}&to={}&period=d&api_token={}&fmt=json".format(ticker,
            date_begin.strftime('%Y-%m-%d'),date_end.strftime('%Y-%m-%d'), eod_key))
        if len(df) == 0:
            print('could not add {}'.format(ticker),end="")
            failed_to_add.append(ticker)
            continue
        df1 = pd.DataFrame(df)
        df1['date'] = pd.to_datetime(df1['date'])
        df1.set_index('date',inplace=True)
        # merge the dataset
        price_df = pd.merge(price_df, df1['adjusted_close'], how='left', left_index=True, right_index=True)
        price_df = price_df.rename(columns={'adjusted_close': ticker})

    # occasionally backup the dataframe
    if count % 20 == 0:
        price_df.to_csv(filename)

  9%|▊         | 70/820 [04:17<27:33,  2.21s/it]  

could not add ACE

 37%|███▋      | 306/820 [15:45<15:31,  1.81s/it]  

could not add RE

 77%|███████▋  | 635/820 [35:01<04:54,  1.59s/it]  

could not add CDAY

 92%|█████████▏| 758/820 [41:31<01:58,  1.91s/it]

could not add LDW

100%|██████████| 820/820 [44:59<00:00,  3.29s/it]
